# DeepPoseKit Step 2 - Annotate your data

This is step 2 of the example notebooks for using DeepPoseKit. This notebook shows you how to annotate your training data with user-defined keypoints using the saved data from step 1.

If you haven't already installed DeepPoseKit you can run the next cell

Use the next cell to download the example data into your home directory

## Enhancements

**This notebook builds on the template provided [here] to**

1. First convert annotation from pre-trained [OpenPose](https://github.com/CMU-Perceptual-Computing-Lab/openpose) model to the required format for the DeepPoseKit toolkit 

2. Cumulate raw images and annotations together to HDF5 format of required batch size to reduce memory usage

### Features to add

1. Get the annotations for a person in an image copied to another person in another image. Say, annotator's 25th frame highlights an annotation - copy this annotation to maybe the next image where the person is in the same (29) position - maybe detect a key input such as - ord('c') ord('2') ord('5') ord(' ') ord('2') ord('9') ord(' ')

2. Extend to 25 annotations

In [ ]:
# !git clone https://github.com/jgraving/deepposekit-data {HOME + '/deepposekit-data'}

In [4]:
import os
from PIL import Image
import numpy as np
import json
import h5py
import time
from copy import deepcopy
from collections import defaultdict

# output should be (num_images, num_coords, 2)
def openpose2deeppose(keypoint, num_points=19, footpoints=False):
    '''
    Returns individual person pose keypoints from openpose json keypoints
    
    openpose keypoints are 
        - 25*(x, y, confidence) with footpoints
        - 19*(x, y, confidence) w/o footpoints
    
    footpoints - if to include heel, bigtoe, smalltoe
    '''
    poses = [data['pose_keypoints_2d'] for data in keypoint['people']]
    only_coords = [[[pose[3*i], pose[3*i+1]] for i in range(num_points)] for pose in poses]
    
    # solution for - images with no annotations doesn't show up in annotator
    if only_coords == []:
        only_coords = [[[0, 0] for i in range(num_points)]]
    
    return only_coords


def get_annotations_image(annotations, person_count, filenames):
    # get list of indices repeated according to person_count
    indices = np.repeat(np.arange(len(person_count)), person_count)
#     print(len(indices), len(annotations))
    assert len(filenames) == len(person_count)
    assert len(annotations) == len(indices)
    
    annotations_consolidated = defaultdict(list)
    
    for index, annotation in zip(indices, annotations):
        annotations_consolidated[filenames[index]].append(list(annotation))
    
    
    return annotations_consolidated

def store_annotations_json(filenames, annotations):
    # get list of indices repeated according to person_count
#     indices = np.repeat(np.arange(len(person_count)), person_count)
    #     print(len(indices), len(annotations))
    assert len(filenames) == len(annotations)

    output_dict = {"name": '', "version":1.3, "people": []}

    set_names = {}
    # create a single json file named after the image name
    # for each annotation of an image add the annotation to "people" using people_dict template
    for name, annotation in zip(filenames, annotations):
        people_dict = {"person_id":[-1],"pose_keypoints_2d":[],"face_keypoints_2d":[],"hand_left_keypoints_2d":[],"hand_right_keypoints_2d":[],"pose_keypoints_3d":[],"face_keypoints_3d":[],"hand_left_keypoints_3d":[],"hand_right_keypoints_3d":[]}

        # To avoid error
        # json.dumps(dicts) - TypeError: Object of type bytes_ is not JSON serializable
        name = name.decode('utf-8')
    #     print(name)
        if name not in set_names.keys():

            new_output_dict = deepcopy(output_dict)

            new_output_dict['name'] = name


            # add 25 annotations
            annotations_mod = np.array(annotation).tolist()
            people_dict["pose_keypoints_2d"] = annotations_mod
#             print(len(annotations), annotations[0].shape)

            new_output_dict['people'].append(people_dict)

            set_names[name] = new_output_dict
        else:
            curr_output_dict = set_names[name]

            people_dict["pose_keypoints_2d"] = np.array(annotation).tolist()
            curr_output_dict['people'].append(people_dict)

    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return json.JSONEncoder.default(self, obj)


    # storing ndarrays as json- https://stackoverflow.com/a/47626762/9734484
    for name, dicts in set_names.items():
        name = name.replace('.jpg', '')
        name = name.replace('.png', '')
        
        with open(f'json/{name}.json', 'w') as fp:
            json.dump(dicts, fp, cls=NumpyEncoder, indent=2)


    print('Store complete')
    
# "output/openpose_annotations_0.h5"
def annotation_from_h5py2json(path):
    print(path)
    with h5py.File(path, 'r') as hf:
        filenames = list(hf['image_path'])
        annotations = list(hf['annotations'])
        
#     print(len(annotations))
#     print(len(filenames))
    store_annotations_json(filenames, annotations)

In [3]:
# len(dicts['people'])
# # dicts['people'][1]
# annotation_string = json.dumps(dicts, indent=2)
# with open(f'json/{name}.json', 'w') as fp:
#     fp.write(annotation_string)
# json.dumps(dicts['name'])
# json.dumps(dicts['people'])
# np.array(annotations).shape
# dicts['people'][0]

### 1. Get image and keypoint, names and paths - change IMAGE_DIR and KEY_DIR

In [39]:
# image directory - raw images - not the openpose image output having keypoints drawn on it
IMAGE_DIR = '/home/nabhanpv/Desktop/projects/mlabs/data/3_23/raw'

# keypoint directory - json keypoint output from openpose
KEY_DIR = '/home/nabhanpv/Desktop/projects/mlabs/data/3_23/output/json_output'


# image file names sorted by date
# eg: ch03_20190823170038_775.jpg, ch03_20190823170038_5275
image_files = [filename for filename in sorted(os.listdir(IMAGE_DIR), 
                                               key=lambda x: 
                                               (int(x.split('_')[1])*10 + int(x.split('_')[2].split('.')[0])))]

# absolute paths to the images
image_paths = [os.path.join(IMAGE_DIR, filename) for filename in image_files]

# keypoint file names corresponding to the image files
# eg: ch03_20190823170038_775_keypoints
keypoint_files = [file.replace('.jpg', '_keypoints.json') for file in image_files]

# path to the keypoint files
keypoint_paths = [os.path.join(KEY_DIR, filename) for filename in keypoint_files]

# loop over the keypoint files and save as dict
keypoint_array = []
for path in keypoint_paths:
    with open(path, 'r') as f:
        keypoint_data = json.load(f)
        keypoint_array.append(keypoint_data)

# change list of lists to np.ndarray
keypoint_array = np.array(keypoint_array)

# store pose keypoints only
keypoints_deeppose = []

# keep count of persons in each image
person_count = []
for keypoint in keypoint_array:
    # extract pose keypoints
    kpoints = openpose2deeppose(keypoint)
    
    # store count of person in each image
    n_persons = len(kpoints)
    person_count.append(n_persons)

    # Add as multiple elements rather than a list
    keypoints_deeppose += kpoints

# cumulative sum of person count
person_count_cum = np.cumsum(person_count)

In [40]:
print(person_count[:15])
# np.array(kpoints).shape
# np.array(openpose2deeppose(keypoint_array[0])).shape

[4, 7, 6, 2, 4, 6, 5, 3, 4, 4, 3, 5, 5, 1, 3]


### 2. Save data into HDF5 - change batch_size and image_per_file

In [82]:
# !rm -rf "./$OUT_DIR"
# !rm -rf "./$META_DIR"
# !rm -rf "./$JSON_DIR"
# !mkdir "./$OUT_DIR"
# !mkdir "./$META_DIR"
# !mkdir "./$JSON_DIR"

import time
import os


# output directory
OUT_DIR = 'output'

# metadata directory - store input image paths
META_DIR = 'metadata'

# json directory
JSON_DIR = 'json'

if not os.path.isdir(OUT_DIR):
    os.mkdir(OUT_DIR)
    
if not os.path.isdir(META_DIR):
    os.mkdir(META_DIR)
    
if not os.path.isdir(JSON_DIR):
    os.mkdir(JSON_DIR)
    

# number of (image,keypoint) pairs to save at a time
BATCH_SIZE = 5

# number of image,keypoint) pairs to save in a hdf5 file
# IMAGE_PER_FILE has to be greater than or equal to BATCH_SIZE
IMAGE_PER_FILE = 5

# if to create a new hdf5 file
INITIALIZE_HDF5 = True


start = time.time()

try:
    count = 50 # set this to starting value of i * BATCH_SIZE to resume from middle files
    hf = None
    
    for i in range(10, 20):#len(image_paths)//BATCH_SIZE + 1):
        print('Opening images')
        # list of image arrays
        image_files_batch = np.array([file for file in image_files[BATCH_SIZE*i : BATCH_SIZE*(i+1)]])
        image_paths_batch = np.array([image_path for image_path in image_paths[BATCH_SIZE*i : BATCH_SIZE*(i+1)]])
        image_array_batch = np.array([np.array(Image.open(image_path)) 
                                for image_path in image_paths_batch])

        '''
        directly indexing keypoints_deeppose will result in lesser keypoints
        there are multiple keypoints corresponding to a single image
        
        person_count = [2, 3, 1, 4, 3, 2, 3, 1, 2]
        person_count_cum = [2, 5, 6, 10, 13, 15, 18, 19, 21]
        
        i = 0
        batch size = 3
        keypoints from 1 to 6
        
        i = 1
        batch size = 3
        keypoints from (6+1) to 15
        
        i = 2
        batch size = 3
        keypoints from (15+1) to 21
        '''
        if i == 0:
            start_idx = 0
            end_idx = person_count_cum[BATCH_SIZE-1] 
        else:
            start_idx = person_count_cum[BATCH_SIZE*i - 1]
            end_idx = person_count_cum[BATCH_SIZE*(i+1) - 1]
            
        keypoints_deeppose_batch = np.array(keypoints_deeppose[start_idx : end_idx])

        person_count_batch = person_count[BATCH_SIZE*i : BATCH_SIZE*(i+1)]

        print(start_idx, end_idx, keypoints_deeppose_batch.shape, person_count_batch)
        
        # repeat image according to the number of person it has
        image_array_repeat_batch = np.repeat(image_array_batch, person_count_batch, axis=0)
        image_files_repeat_batch = np.repeat(image_files_batch, person_count_batch, axis=0)

        # save images and keypoints
        if ((count) % IMAGE_PER_FILE) == 0:
            # close opened file
            if hf:
                with open(f'{META_DIR}/metadata_annotations_{file_index}.txt', 'a') as f:
                    f.write(str(list((hf["image_path"]))))
                    
                    
#                 annotations_consolidated = get_annotations_image(hf['annotations'], 
#                                                                  person_count[count-IMAGE_PER_FILE : count], 
#                                                                  image_files_batch[count-IMAGE_PER_FILE : count])
                
#                 with open(f'{JSON_DIR}/annotations_{file_index}.txt', 'a') as f:
#                     f.write(str(annotations_consolidated))
#                     f.write(json.dumps(str(list((hf["annotations"])))))
#                     f.write(str(list((hf["annotations"]))))
                    
                hf.close()
                
            print('Creating a new hdf5 file')
            
            # open file for saving data
            file_index = count // IMAGE_PER_FILE
            hf = h5py.File(f'{OUT_DIR}/openpose_annotations_{file_index}.h5', 'a')
            
            # maxshape should be (None,) to be able to extend the file
            # chunks let the file chunks be stored in diff parts of the disk
            hf.create_dataset('images', data=image_array_repeat_batch, compression='gzip', compression_opts=9, 
                              chunks=True, maxshape=(None,*image_array_repeat_batch.shape[1:]))
            hf.create_dataset('annotations', data=keypoints_deeppose_batch, compression='gzip', compression_opts=9, 
                              chunks=True, maxshape=(None,*keypoints_deeppose_batch.shape[1:]))
            
            # hdf5 can't store list of strings 
            path_list = np.array(image_files_repeat_batch, dtype='S')
            hf.create_dataset('image_path', data=path_list, compression='gzip', compression_opts=9, 
                              chunks=True, maxshape=(None,))
#             print(path_list)
        else:
            print('Extending hdf5 file with new data')
            
            hf["images"].resize((hf["images"].shape[0] + image_array_repeat_batch.shape[0]), axis = 0)
            hf["images"][-image_array_repeat_batch.shape[0]:] = image_array_repeat_batch

            hf["annotations"].resize((hf["annotations"].shape[0] + keypoints_deeppose_batch.shape[0]), axis = 0)
            hf["annotations"][-keypoints_deeppose_batch.shape[0]:] = keypoints_deeppose_batch
                              
            # hdf5 can't store list of strings 
            #path_list = np.char.encode(np.array(image_paths_batch, dtype='U'), encoding='utf8')
            path_list = np.array(image_files_repeat_batch, dtype='S')
            hf["image_path"].resize((hf["image_path"].shape[0] + path_list.shape[0]), axis = 0)
            hf["image_path"][-path_list.shape[0]:] = path_list
                
        count += BATCH_SIZE
        print(f"{i:} {hf['images'].shape} {hf['annotations'].shape} {hf['image_path'].shape}\n\n")
              
    # save data on exit
    if hf:
        with open(f'{META_DIR}/metadata_annotations_{file_index}.txt', 'a') as f:
            print('Writing metadata')
            f.write(str(list((hf["image_path"]))))

#         annotations_consolidated = get_annotations_image(list(hf['annotations']), 
#                                                              person_count[count-BATCH_SIZE : count], 
#                                                              image_files_batch[count-BATCH_SIZE : count])

#         with open(f'{JSON_DIR}/annotations_{file_index}.txt', 'a') as f:
#             print('writing json')
#             f.write(str(annotations_consolidated))

        hf.close()
except Exception as e:
    print(e)
finally:
    if hf:
        hf.close()
    print(f"Completed in {time.time()-start}")

Opening images
167 184 (17, 19, 2) [3, 1, 1, 7, 5]
Creating a new hdf5 file
10 (17, 1080, 1920, 3) (17, 19, 2) (17,)


Opening images
184 211 (27, 19, 2) [7, 9, 4, 4, 3]
Creating a new hdf5 file
11 (27, 1080, 1920, 3) (27, 19, 2) (27,)


Opening images
211 223 (12, 19, 2) [3, 1, 3, 3, 2]
Creating a new hdf5 file
12 (12, 1080, 1920, 3) (12, 19, 2) (12,)


Opening images
223 244 (21, 19, 2) [5, 5, 5, 4, 2]
Creating a new hdf5 file
13 (21, 1080, 1920, 3) (21, 19, 2) (21,)


Opening images
244 262 (18, 19, 2) [2, 4, 3, 6, 3]
Creating a new hdf5 file
14 (18, 1080, 1920, 3) (18, 19, 2) (18,)


Opening images
262 285 (23, 19, 2) [6, 4, 5, 4, 4]
Creating a new hdf5 file
15 (23, 1080, 1920, 3) (23, 19, 2) (23,)


Opening images
285 296 (11, 19, 2) [2, 1, 3, 4, 1]
Creating a new hdf5 file
16 (11, 1080, 1920, 3) (11, 19, 2) (11,)


Opening images
296 312 (16, 19, 2) [3, 2, 3, 5, 3]
Creating a new hdf5 file
17 (16, 1080, 1920, 3) (16, 19, 2) (16,)


Opening images
312 326 (14, 19, 2) [2, 1, 3, 4, 

Annotation Hotkeys
------------
* <kbd>+</kbd><kbd>-</kbd> = rescale image by ±10%
* <kbd>left mouse button</kbd> = move active keypoint to cursor location
* <kbd>W</kbd><kbd>A</kbd><kbd>S</kbd><kbd>D</kbd> = move active keypoint 1px or 10px
* <kbd>space</kbd> = change <kbd>W</kbd><kbd>A</kbd><kbd>S</kbd><kbd>D</kbd> mode (swaps between 1px or 10px movements)
* <kbd>J</kbd><kbd>L</kbd> = next or previous image
* <kbd><</kbd><kbd>></kbd> = jump 10 images forward or backward
* <kbd>I</kbd>,<kbd>K</kbd> or <kbd>tab</kbd>, <kbd>shift</kbd>+<kbd>tab</kbd> = switch active keypoint
* <kbd>R</kbd> = mark image as unannotated ("reset")
* <kbd>F</kbd> = mark image as annotated ("finished")
* <kbd>esc</kbd> or <kbd>Q</kbd> = quit

# Annotate data
Annotations are saved automatically. 
The skeleton in each frame will turn blue when the frame is fully annotated. If there are no visible keypoints, this means the frame hasn't been annotated, so try clicking to position the keypoint in the frame.

In [2]:
%load_ext autoreload
%autoreload 2

from deepposekit import Annotator
from os.path import expanduser
import glob
HOME = expanduser("~")

### 3. Run annotator - change datapath argument

In [100]:
import cv2


print(n)
datapath = f'output/openpose_annotations_{n}.h5'
app = Annotator(datapath=datapath,
            dataset='images',
            skeleton='skeleton.csv',
            shuffle_colors=False,
            text_scale=1.0)

n += 1

'''
print statement in /home/nabhanpv/miniconda3/lib/python3.7/site-packages/deepposekit-0.3.5-py3.7.egg/deepposekit/annotate/gui/Annotator.py

print((self.n_images, self.n_keypoints))

print(self.skeleton.loc[:, ["x", "y"]].shape)
print(h5file["annotations"][self.image_idx].shape)

print(self.skeleton.loc[:, "annotated"].shape)
print(h5file["annotated"][self.image_idx].shape)

print(h5file['annotated'].shape)
'''

# app = Annotator(datapath=HOME + '/deepposekit-data/datasets/locust/annotation_data_release.h5',
#                 dataset='images',
#                 skeleton=HOME + '/deepposekit-data/datasets/locust/skeleton.csv',
#                 shuffle_colors=False,
#                 text_scale=0.2)

13


'\nprint statement in /home/nabhanpv/miniconda3/lib/python3.7/site-packages/deepposekit-0.3.5-py3.7.egg/deepposekit/annotate/gui/Annotator.py\n\nprint((self.n_images, self.n_keypoints))\n\nprint(self.skeleton.loc[:, ["x", "y"]].shape)\nprint(h5file["annotations"][self.image_idx].shape)\n\nprint(self.skeleton.loc[:, "annotated"].shape)\nprint(h5file["annotated"][self.image_idx].shape)\n\nprint(h5file[\'annotated\'].shape)\n'

In [101]:
start = time.time()

app.run()

end = time.time()

print((end-start)/60)

Creating new annotation
Annotator._new_annotation(): new image added (22, 1080, 1920, 3)
Annotator._new_annoation(): new annotation added (22, 19, 2)
Annotator._new_annoation(): new annotation added (22, 19)
Successfully created new annotation
Creating new annotation
Annotator._new_annotation(): new image added (23, 1080, 1920, 3)
Annotator._new_annoation(): new annotation added (23, 19, 2)
Annotator._new_annoation(): new annotation added (23, 19)
Successfully created new annotation
Creating new annotation
Annotator._new_annotation(): new image added (24, 1080, 1920, 3)
Annotator._new_annoation(): new annotation added (24, 19, 2)
Annotator._new_annoation(): new annotation added (24, 19)
Successfully created new annotation
Creating new annotation
Annotator._new_annotation(): new image added (25, 1080, 1920, 3)
Annotator._new_annoation(): new annotation added (25, 19, 2)
Annotator._new_annoation(): new annotation added (25, 19)
Successfully created new annotation
Creating new annotation


### 4. H5PY to JSON - change function input

In [102]:
annotation_from_h5py2json(datapath)

output/openpose_annotations_13.h5
Store complete


<hr>

In [53]:
import h5py
idx = 1
# # ['annotated', 'annotations', 'image_path', 'images', 'skeleton']>
with h5py.File("output/openpose_annotations_3.h5", 'r+') as hf:
    print(list(hf['image_path']))
    print(hf['images'].shape)
    print(hf['image_path'].shape)
    print(hf['annotations'].shape)
    print(hf['annotated'].shape)
#     hf['image_path'][:] = p
#     print(set(list(hf['image_path'])))
#     print(hf['annotations'].shape)
#     hf['images'][idx:-1] = hf['images'][idx+1:]
#     hf['images'].resize((hf[self.dataset].shape[0] - 1), axis = 0)
#     print(hf['images'].shape)
#     print(list(hf['image_path']))
# #     print(hf['skeleton'].shape)
#     print(hf.keys())
# #     print(hf['images'].shape)

[b'ch03_20190823171617_159.jpg', b'ch03_20190823171617_159.jpg', b'ch03_20190823171617_1059.jpg', b'ch03_20190823171617_1959.jpg', b'ch03_20190823171617_1959.jpg', b'ch03_20190823171617_1959.jpg', b'ch03_20190823171925_124.jpg', b'ch03_20190823171925_124.jpg', b'ch03_20190823172024_466.jpg', b'ch03_20190823172024_466.jpg', b'ch03_20190823172024_466.jpg', b'ch03_20190823172024_466.jpg', b'ch03_20190823172024_466.jpg', b'ch03_20190823172024_466.jpg']
(14, 1080, 1920, 3)
(14,)
(14, 19, 2)
(14, 19)


In [39]:
# image_files[:6]

path = ['ch03_20190823170038_775.jpg',
'ch03_20190823170038_1675.jpg',
'ch03_20190823170038_2575.jpg',
'ch03_20190823170038_3475.jpg',
'ch03_20190823170038_4375.jpg']

pc = [6, 10, 8, 7, 9]

p = np.array(np.repeat(path, pc, axis=0), dtype='S')
# p

In [114]:
# os.listdir('json')

['ch03_20190828191227_3587_rendered.json',
 'ch03_20190828191227_2687_rendered.json',
 'ch03_20190828191722_67_rendered.json',
 'ch03_20190828191227_887_rendered.json']

In [133]:
# with open('json/ch03_20190828191227_2687_rendered.json') as f:
#     j = json.load(f)

In [134]:
# np.array(j['people']).shape

(4,)

In [93]:
n = 11